# Get Embeddings

In [0]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set up Azure OpenAI

In [0]:
!pwd

/Workspace/Repos/jeanpierre.vanniekerk@truenorthgroup.co.za/document-analysis-using-gpt-3.ide/notebooks


In [0]:
!pip install -r /Workspace/Repos/jeanpierre.vanniekerk@truenorthgroup.co.za/document-analysis-using-gpt-3.ide/notebooks/requirements.txt

  Using cached azure_identity-1.6.0-py2.py3-none-any.whl (108 kB)
  Using cached streamlit-1.18.1-py2.py3-none-any.whl (9.6 MB)
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached matplotlib-3.6.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached plotly-5.12.0-py2.py3-none-any.whl (15.2 MB)
  Using cached scipy-1.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached scikit_learn-1.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.5 MB)
  Using cached tiktoken-0.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached llama_index-0.4.33-py3-none-any.whl
  Using cached langchain-0.0.129-py3-none-any.whl (467 kB)
  Using cached faiss_cpu-1.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Using cached msal_extensions-0.3.1-py2.py3-none-any.whl (18 kB)
  Using cached msal-1.2

In [0]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

False

## Load Data

In [0]:
import pandas as pd
# File location and type
file_location = "/FileStore/tables/bbc_news_data_embedding.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = "\t"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format('com.databricks.spark.csv') \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

temp_table_name = "bbc_news_data_embedding_csv"

df.createOrReplaceTempView(temp_table_name)
df = df.toPandas()

## Deploy a model

In [0]:
# list models deployed with embeddings capability
deployment_id = None
result = openai.Deployment.list()

for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    if model["capabilities"]["embeddings"] != True:
        continue
    
    deployment_id = deployment["id"]
    break

# if not model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')
    model = "text-similarity-davinci-002"

    # Now let's create the deployment
    print(f'Creating a new deployment with model: {model}')
    result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {model} with deployment_id {deployment_id}')
else:
    print(f'Found a succeeded deployment that supports embeddings with id: {deployment_id}.')

Found a succeeded deployment that supports embeddings with id: text-embedding-ada-002.


## Get Embeddings
ref: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/tutorials/embeddings?tabs=bash

In [0]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")
df1 = df
print(len(df1))
df1['n_tokens'] = df1['content'].apply(lambda x: len(tokenizer.encode(x)))
df1 = df1[df1.n_tokens<8192]
print(len(df1))
df1

2225
2225


,category,filename,title,content,embedding,n_tokens
0,business,001.txt,Ad sales boost Time Warner profit,""" Quarterly profits at US media giant TimeWarn...","[-0.0012276918860152364, 0.00733763724565506, ...",563
1,business,002.txt,Dollar gains on Greenspan speech,""" The dollar has hit its highest level against...","[0.0009311728645116091, 0.014099937863647938, ...",457
2,business,003.txt,Yukos unit buyer faces loan claim,""" The owners of embattled Russian oil giant Yu...","[-0.010487922467291355, 0.009665092453360558, ...",340
3,business,004.txt,High fuel prices hit BA's profits,""" British Airways has blamed high fuel prices ...","[0.0111119095236063, 0.004624682944267988, -0....",558
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,"[-0.0021637482568621635, 0.005410161800682545,...",385
...,...,...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,""" BT is introducing two initiatives to help be...","[0.007671569474041462, 0.00624304823577404, -0...",502
2221,tech,398.txt,Spam e-mails tempt net shoppers,""" Computer users across the world continue to ...","[0.0026338498573750257, 0.015989987179636955, ...",435
2222,tech,399.txt,Be careful how you code,""" A new European directive could put software ...","[0.007126151118427515, 0.008495588786900043, -...",1208
2223,tech,400.txt,US cyber security chief resigns,""" The man making sure US computer networks are...","[0.002447678940370679, 0.006076449993997812, -...",437


In [0]:
df['embedding'] = ''
print(len(df))
for i in range(len(df)):    
    try:
        embedding = openai.Embedding.create(input=df['content'][i], deployment_id=deployment_id)
        df['embedding'][i] = embedding['data'][0]['embedding']
    except Exception as err:
        i
        print(f"Unexpected {err=}, {type(err)=}")

2225


<command-1550531866917453>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedding'][i] = embedding['data'][0]['embedding']


In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS OpenAI")
df_sp = spark.createDataFrame(df)
df_sp.write.mode('overwrite').option("overwriteSchema", "true").saveAsTable('openai.document_analysis_embeddings')

DataFrame[]

In [0]:
sdf = spark.sql('select * from openai.document_analysis_embeddings')
sdf.limit(10).toPandas()

,category,filename,title,content,embedding,n_tokens
0,business,001.txt,Ad sales boost Time Warner profit,""" Quarterly profits at US media giant TimeWarn...","[-0.01796060986816883, -0.01651429571211338, 0...",563
1,business,002.txt,Dollar gains on Greenspan speech,""" The dollar has hit its highest level against...","[-0.021637994796037674, -0.012706875801086426,...",457
2,business,003.txt,Yukos unit buyer faces loan claim,""" The owners of embattled Russian oil giant Yu...","[-0.01746140420436859, -0.03939249739050865, 0...",340
3,business,004.txt,High fuel prices hit BA's profits,""" British Airways has blamed high fuel prices ...","[-0.019902830943465233, -0.017122669145464897,...",558
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,"[-0.008347506634891033, -0.008401579223573208,...",385
5,business,006.txt,Japan narrowly escapes recession,""" Japan's economy teetered on the brink of a t...","[-0.023057539016008377, -0.011685534380376339,...",232
6,business,007.txt,Jobs growth still slow in the US,""" The US created fewer jobs than expected in J...","[-0.04825970157980919, -0.019267477095127106, ...",348
7,business,008.txt,India calls for fair trade rules,""" India, which attends the G7 meeting of seven...","[-0.01497228816151619, -0.017229484394192696, ...",404
8,business,009.txt,Ethiopia's crop production up 24%,""" Ethiopia produced 14.27 million tonnes of cr...","[-0.007256364915519953, -0.0311115849763155, -...",310
9,business,010.txt,Court rejects $280bn tobacco case,A US government claim accusing the country's ...,"[-0.008582530543208122, -0.00982870627194643, ...",282


## Save data

In [0]:
#df.to_csv("../data/bbc-news-data-embedding.csv", sep='\t', index=False)